In [30]:
!pip install langchain cohere faiss-cpu youtube-transcript-api

In [31]:
import os
os.environ["COHERE_API_KEY"] = "iOBYyDUk3G0NGsnD1BkbVNS8tueEeCVc0y2Vfg5V"

In [33]:
# ✅ Step 2: Load Your YouTube Transcript (manually paste it here as a string)
text = """
My friend Mac is about to experience the scariest moment of his life.
I'm just going to let you know I'm not afraid of anything.
- We'll see. - You cannot scare me.
Seal him up.
You have to survive 10 minutes with a thousand spiders to move on.
Release the spiders.
Start the timer.
The reason we're doing this is because
Mack said no matter how hard I tried, I can't scare him.
Would you say you're scared now?
Getting there a little bit.
- A little bit? - Yeah.
And this is only one of many fears Mack will face in this video.
Oh, good.
And if he can successfully face all of the fears, he will win $800,000.
I'm not going to lie,
this was way more gnarly than I thought it would be.
Three, two, one.
Lift it up. Lift it up.
Oh yeah.
Congratulations on passing the first challenge.
Let's go.
But Mack has to finish all of these fears
if he wants to win the $800,000 prize.
Mack, I know you're blindfolded, but trust me.
Just walk up this ramp.
Raise us up.
Why are you taking a knee?
What is...?
Stand up. Stand up. You're fine. You're fine.
"""

In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.vectorstores import FAISS

In [36]:
pip install -U langchain-cohere


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.7 MB/s eta 0:00:00


In [47]:
#  Step 3: Create Chunks and Generate Embeddings using Cohere
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_cohere import CohereEmbeddings, ChatCohere
from langchain_community.vectorstores import FAISS

# 1. Split the text
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.create_documents([text])

# 2. Generate embeddings and store in FAISS
embeddings = CohereEmbeddings(model="embed-english-v3.0")
vector_store = FAISS.from_documents(chunks, embeddings)

In [48]:
#  Step 4: Setup Retriever
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

#  Step 5: Prompt Template Setup
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables=['context', 'question']
)


In [49]:
llm = ChatCohere(model="command-r-plus")

In [50]:
#  Step 7: Retrieval + Prompting Chain
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

def format_docs(retrieved_docs):
    return "\n\n".join(doc.page_content for doc in retrieved_docs)

parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

parser = StrOutputParser()
main_chain = parallel_chain | prompt | llm | parser

In [53]:
#  Step 8: Ask a Question
query = "what type of video is this, and explain it"
print("\nAnswer:\n", main_chain.invoke(query))


Answer:
 This appears to be a video of a person named Mack undergoing a series of challenges designed to face their fears. The video seems to be a social experiment or a competition, with a potential prize of $800,000 if Mack successfully confronts all the fears presented to him. In the provided transcript, Mack is faced with their fear of spiders, as they are sealed in a room with a thousand spiders for 10 minutes. The video seems to be an intense and emotional journey for Mack, as they attempt to overcome their fears and work towards a significant monetary reward.


In [54]:
print(main_chain.invoke("Can you summarize the text?"))

Mack is about to undergo a series of scary challenges to prove that he cannot be scared. His friend doubts this claim and has set up these challenges, with the incentive of winning $800,000 if he faces and overcomes all of his fears. The first challenge involves surviving in an enclosed space with a thousand spiders for 10 minutes, which Mack manages to pass, despite his growing fear.


In [55]:
pip install gradio langchain cohere faiss-cpu


In [58]:
import gradio as gr
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_cohere.embeddings import CohereEmbeddings
from langchain_cohere.chat_models import ChatCohere

# ---- Setup prompt template ----
prompt = PromptTemplate(
    template="""
You are a helpful assistant.
Answer ONLY from the provided transcript context.
If the context is insufficient, just say you don't know.

{context}
Question: {question}
""",
    input_variables=["context", "question"]
)

# ---- Function to build the main_chain from a transcript ----
def build_qa_pipeline(transcript_text):
    # Split
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    docs = splitter.create_documents([transcript_text])

    # Embeddings + Vector store
    embeddings = CohereEmbeddings(model="embed-english-v3.0")
    vector_store = FAISS.from_documents(docs, embeddings)
    retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

    # LLM
    llm = ChatCohere(model="command-r-plus")  # ✅ Updated class usage

    # Chain setup
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    parallel_chain = RunnableParallel({
        'context': retriever | RunnableLambda(format_docs),
        'question': RunnablePassthrough()
    })

    output_parser = StrOutputParser()

    return parallel_chain | prompt | llm | output_parser

# ---- Gradio Interface logic ----
def run_qa(transcript, question):
    try:
        qa_chain = build_qa_pipeline(transcript)
        answer = qa_chain.invoke(question)
        return answer
    except Exception as e:
        return f"[ERROR] {str(e)}"

# ---- Gradio App Layout ----
with gr.Blocks() as demo:
    gr.Markdown("## 🎥 YouTube Transcript QA Chatbot (Cohere + LangChain)")

    transcript_input = gr.Textbox(lines=10, label="Paste Full Transcript")
    question_input = gr.Textbox(label="Ask a Question")
    submit_btn = gr.Button("Submit")
    answer_output = gr.Textbox(label="Answer")

    submit_btn.click(fn=run_qa, inputs=[transcript_input, question_input], outputs=answer_output)

# ---- Launch the App ----
demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://eb0a4ca25b4348c8ee.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
